<a href="https://colab.research.google.com/github/vaibhavikharkar/MY-PROJECT/blob/main/Hybrid_Recommender_%26_Cement_Quality_Prediction_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Connect to SQLite database
conn = sqlite3.connect("hybrid_system.db")
cursor = conn.cursor()

# Create tables
cursor.executescript('''
    CREATE TABLE IF NOT EXISTS books (
        BookID INTEGER PRIMARY KEY,
        Title TEXT,
        Genre TEXT,
        Popularity INTEGER
    );

    CREATE TABLE IF NOT EXISTS user_ratings (
        UserID INTEGER,
        BookID INTEGER,
        Rating INTEGER,
        FOREIGN KEY(BookID) REFERENCES books(BookID)
    );

    CREATE TABLE IF NOT EXISTS cement_data (
        CementID INTEGER PRIMARY KEY,
        CementComposition TEXT,
        Strength REAL
    );
''')
conn.commit()

# Sample data insertion
cursor.executescript('''
    INSERT INTO books (BookID, Title, Genre, Popularity)
    VALUES (1, 'Python Programming', 'Education', 500),
           (2, 'Machine Learning Basics', 'Technology', 400),
           (3, 'Data Science Handbook', 'Science', 350);

    INSERT INTO user_ratings (UserID, BookID, Rating)
    VALUES (1, 1, 5), (2, 1, 4), (1, 2, 3), (3, 2, 5), (3, 3, 4);

    INSERT INTO cement_data (CementID, CementComposition, Strength)
    VALUES (1, 'Limestone 50%, Clay 30%, Iron 10%, Gypsum 10%', 42.5),
           (2, 'Limestone 60%, Clay 25%, Iron 5%, Gypsum 10%', 48.3);
''')
conn.commit()

# Collaborative Filtering for Book Recommendation
ratings_df = pd.read_sql_query("SELECT * FROM user_ratings", conn)
books_df = pd.read_sql_query("SELECT * FROM books", conn)

book_pivot = ratings_df.pivot(index='UserID', columns='BookID', values='Rating').fillna(0)
cos_sim = cosine_similarity(book_pivot)

book_sim_df = pd.DataFrame(cos_sim, index=book_pivot.index, columns=book_pivot.index)
print("Book Similarity Matrix:\n", book_sim_df)

# Predicting Cement Strength
cement_df = pd.read_sql_query("SELECT * FROM cement_data", conn)
vectorizer = TfidfVectorizer()
X_cement = vectorizer.fit_transform(cement_df['CementComposition'])
y_cement = cement_df['Strength']

X_train, X_test, y_train, y_test = train_test_split(X_cement, y_cement, test_size=0.2, random_state=42)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print("Cement Strength Prediction MSE:", mean_squared_error(y_test, y_pred))

# Close connection
conn.close()




Book Similarity Matrix:
 UserID         1         2         3
UserID                              
1       1.000000  0.857493  0.401754
2       0.857493  1.000000  0.000000
3       0.401754  0.000000  1.000000
Cement Strength Prediction MSE: 33.639999999999965
